## This notebook aims to develope the Accuracy testing

In [1]:
import numpy as np 
from tools.GH_Quadrature import*
from tools.Ord_Herm_Pol_1 import*
from tools.Num_Stab_Approx import*

In [ ]:
def Accuracy_Test_1(sigma,rho,beta,gam,alpha,delta,k,a,bk,D,IM,PF,zb,discard):
    	


In [ ]:
for p in range(1, a.shape[0]+1):#need to check if we need to add 1 or not
      

Within the for loop

In [ ]:
k0 = 
a0 =
k1 = Ord_Herm_Pol_1()
c0 = 
a1 = np.power(a0, rho)*np.exp(epsi_nodes)
k1_dupl = np.asarray([k1]*n_nodes).reshape(n_nodes, 1)
X1 = Ord_Herm_Pol_1()
k2 = X1 bk


Testing area: